In [752]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

Cargo los dataframes originales

In [753]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

Me quedo con las columnas de interes

In [754]:
df_reducido = df_original.loc[:,['person','event','sku','model','city','region','country']]
df_reducido.head()

,person,event,sku,model,city,region,country
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN


Construyo un dataframe que tiene las personas, con las regiones a las que pertenecen, y solo las que realizaron viewed product

In [755]:
df_region_person = df_original.loc[df_original['country'].notnull(),['person','region','country']]

In [756]:
df_region_person['counter'] = 1
df_prueba = df_region_person.groupby(['person','region'])['counter'].sum().reset_index()

In [757]:
df_region_person= df_prueba.sort_values(by='counter',ascending = False).drop_duplicates(subset='person',keep='first')

In [758]:
df_region_person = df_region_person.rename(columns={'region':'most_visited_region'})

In [759]:
df_region_person = df_region_person.drop(['counter'],axis=1)

In [760]:
df_reducido = df_reducido.merge(df_region_person, left_on='person', right_on='person', how='left')
df_reducido.head()

,person,event,sku,model,city,region,country,most_visited_region
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,NaN,NaN,NaN,Rio de Janeiro
1,ad93850f,viewed product,304.0,iPhone 5s,NaN,NaN,NaN,Sao Paulo
2,0297fc1e,viewed product,6888.0,iPhone 6S,NaN,NaN,NaN,Rio de Janeiro
3,2d681dd8,viewed product,11890.0,iPhone 7,NaN,NaN,NaN,Sao Paulo
4,cccea85e,viewed product,7517.0,LG G4 H818P,NaN,NaN,NaN,Sao Paulo


In [761]:
df_reducido = df_reducido.loc[df_reducido['sku'].notnull(),['person','event','sku','model','most_visited_region']]
df_reducido.head()

,person,event,sku,model,most_visited_region
0,4886f805,viewed product,9288.0,Samsung Galaxy J7 Prime,Rio de Janeiro
1,ad93850f,viewed product,304.0,iPhone 5s,Sao Paulo
2,0297fc1e,viewed product,6888.0,iPhone 6S,Rio de Janeiro
3,2d681dd8,viewed product,11890.0,iPhone 7,Sao Paulo
4,cccea85e,viewed product,7517.0,LG G4 H818P,Sao Paulo


Ahora tengo un dataframe que tiene por persona, la region a la que pertenece y los modelos con los que interactuo. Ahora, solo quiero los que miro:

In [762]:
df_only_vp = df_reducido.loc[df_reducido['event'] == 'viewed product', :]

Ahora quiero, por persona, el producto que mas visito:

In [763]:
df_most_visited_per_person = df_only_vp.groupby(['person','model']).size().reset_index()\
                                                                            .rename(columns={0:'Cantidad_visitas'})

In [764]:
df_most_visited_per_person = df_most_visited_per_person.sort_values(by='Cantidad_visitas',ascending=False)\
                                                        .drop_duplicates(subset='person',keep='first')

In [765]:
df_most_visited_per_person = df_most_visited_per_person.merge(df_region_person, on='person',how='left')

In [766]:
df_most_visited_per_person.head()

,person,model,Cantidad_visitas,most_visited_region
0,b1f4dbf6,iPhone 5s,832,Sao Paulo
1,171e75cb,iPhone 6S,679,Espirito Santo
2,6f19cfd9,iPhone 6S Plus,622,Bahia
3,6abd2bf1,iPhone 6,603,Sao Paulo
4,9bf968c5,iPhone 6,558,Bahia


Ahora, un dataframe que tiene el top 5 de cada region de los productos mas visitados.

In [767]:
df_most_visited_per_region = df_original.loc[df_original['event'] == 'viewed product', ['person','region', 'model', 'sku']]
df_most_visited_per_region = df_most_visited_per_region.drop(columns='region')
df_most_visited_per_region = df_most_visited_per_region.merge(df_region_person, on='person', how='left')
df_most_visited_per_region = df_most_visited_per_region.rename(columns={'most_visited_region':'region'})
df_most_visited_per_region.head()

,person,model,sku,region
0,4886f805,Samsung Galaxy J7 Prime,9288.0,Rio de Janeiro
1,ad93850f,iPhone 5s,304.0,Sao Paulo
2,0297fc1e,iPhone 6S,6888.0,Rio de Janeiro
3,2d681dd8,iPhone 7,11890.0,Sao Paulo
4,cccea85e,LG G4 H818P,7517.0,Sao Paulo


Aca calculo la cantidad de visitas por producto y por regions

In [768]:
df_most_visited_per_region = df_most_visited_per_region.groupby(['region','model']).size().reset_index().rename(columns={0:'Visitas'})

In [769]:
df_most_visited_per_region.sort_values(by='Visitas',ascending=False).head()

,region,model,Visitas
4358,Sao Paulo,iPhone 6,30010
4360,Sao Paulo,iPhone 6S,27908
4357,Sao Paulo,iPhone 5s,26446
4362,Sao Paulo,iPhone 7,18638
4955,Unknown,iPhone 6,15574


Ahora, solo conservamos el top 5 de cada una de las regiones

In [770]:
df_most_visited_per_region = df_most_visited_per_region.sort_values(by='Visitas',ascending=False)\
                                                       .groupby('region').head(5)

Construyo un diccionario que tiene como clave la region, y como valor una lista con los 5 productos mas visitados por los usuarios de esa region (algunas listas tienen menos de 5 items por la falta de datos de algunas regiones)

In [771]:
df_top_5_por_region = df_most_visited_per_region.groupby('region')
top_5_region = {}
regiones_del_df = df_most_visited_per_region['region'].drop_duplicates().dropna()
for region in regiones_del_df:
    top_5_region[region] = list(df_top_5_por_region.get_group(region)['model'])

In [772]:
top_5_region['Sao Paulo']

['iPhone 6', 'iPhone 6S', 'iPhone 5s', 'iPhone 7', 'Samsung Galaxy S7 Edge']

Ahora, utilizando ese diccionario y el dataframe del producto mas visitado por cada persona, construyo una columna que decide si la persona visitó mas veces un producto que entra en el top 5 de su region.

In [773]:
df_most_visited_per_person = df_most_visited_per_person.dropna()
df_most_visited_per_person['person'].describe()


count        36931
unique       36931
top       1cfa87f1
freq             1
Name: person, dtype: object

In [774]:
df_most_visited_per_person['busco_top_5_visitas'] = [s.model in top_5_region[s.most_visited_region] for s in df_most_visited_per_person.itertuples()]

In [775]:
df_most_visited_per_person.head(10)

,person,model,Cantidad_visitas,most_visited_region,busco_top_5_visitas
0,b1f4dbf6,iPhone 5s,832,Sao Paulo,True
1,171e75cb,iPhone 6S,679,Espirito Santo,True
2,6f19cfd9,iPhone 6S Plus,622,Bahia,True
3,6abd2bf1,iPhone 6,603,Sao Paulo,True
4,9bf968c5,iPhone 6,558,Bahia,True
5,656c18ef,iPhone 6S,533,Unknown,True
6,eb6c498c,iPhone 6S,500,Minas Gerais,True
7,0d5f66ed,iPhone 5s,495,Alagoas,True
8,0dbcf50c,iPhone SE,486,Unknown,False
9,97b0c0d1,iPhone SE,477,Sao Paulo,False


In [776]:
df_final = df_most_visited_per_person

Voy a hacer un df que tiene la cantidad de visitas de una misma persona, en su region, por modelo que haya visitado

In [777]:
df_cantidad_visitas = df_original.loc[df_original['event'] == 'viewed product', ['person','model','event']]
df_cantidad_visitas = df_cantidad_visitas.merge(df_region_person, on='person', how='left')
df_cantidad_visitas = df_cantidad_visitas.groupby(['most_visited_region','person','model']).size().reset_index()
df_cantidad_visitas.loc[df_cantidad_visitas['most_visited_region'] == 'Sao Paulo',:].sort_values(by=0,ascending=False).head()

,most_visited_region,person,model,0
162853,Sao Paulo,b1f4dbf6,iPhone 5s,832
145868,Sao Paulo,6abd2bf1,iPhone 6,603
162858,Sao Paulo,b1f4dbf6,iPhone SE,554
145870,Sao Paulo,6abd2bf1,iPhone 6S,514
156495,Sao Paulo,97b0c0d1,iPhone SE,477


Y ahora, con este df calculo cuanto es el promedio de visitas de un modelo por region

In [778]:
df_promedio_de_visitas = df_cantidad_visitas.groupby(['most_visited_region','model'])[0].mean().reset_index()
df_promedio_de_visitas.loc[df_promedio_de_visitas['most_visited_region'] == 'Sao Paulo',:].sort_values(by=0,ascending=False).head()

,most_visited_region,model,0
4319,Sao Paulo,Samsung Galaxy Young 2 Duos TV,11.000000
4360,Sao Paulo,iPhone 6S,10.440703
4357,Sao Paulo,iPhone 5s,9.916010
4358,Sao Paulo,iPhone 6,9.829676
4301,Sao Paulo,Samsung Galaxy S7 Edge,9.117329


In [779]:
df_promedio_de_visitas = df_promedio_de_visitas.rename(columns={0:'Promedio_visitas_producto'})

Mergeo esa informacion en el dataframe final

In [780]:
df_final = df_final.merge(df_promedio_de_visitas, on=['most_visited_region','model'], how='left')
df_final.head()

,person,model,Cantidad_visitas,most_visited_region,busco_top_5_visitas,Promedio_visitas_producto
0,b1f4dbf6,iPhone 5s,832,Sao Paulo,True,9.916010
1,171e75cb,iPhone 6S,679,Espirito Santo,True,13.250000
2,6f19cfd9,iPhone 6S Plus,622,Bahia,True,11.249071
3,6abd2bf1,iPhone 6,603,Sao Paulo,True,9.829676
4,9bf968c5,iPhone 6,558,Bahia,True,9.791096


Nuevo feature: ver si la persona visito el producto mas veces que el promedio

In [781]:
df_final['visito_mas_que_el_promedio'] = df_final['Cantidad_visitas'] > df_final['Promedio_visitas_producto']
df_final['visito_mas_que_el_promedio'].value_counts()

False    19277
True     17654
Name: visito_mas_que_el_promedio, dtype: int64

Ahora, saco cuanto tiempo gasto una persona en la pagina visitando productos

In [782]:
df_tiempo_por_persona = df_original.loc[df_original['event'] == 'viewed product',['person','timestamp']]

In [783]:
df_tiempo_por_persona['dia'] = pd.to_datetime(df_tiempo_por_persona['timestamp']).dt.day
df_tiempo_por_persona['mes'] = pd.to_datetime(df_tiempo_por_persona['timestamp']).dt.month
df_tiempo_por_persona.head(2)

,person,timestamp,dia,mes
0,4886f805,2018-05-18 00:11:59,18,5
1,ad93850f,2018-05-18 00:11:27,18,5


In [784]:
df_tiempo_por_persona['DeltaTime'] = pd.to_datetime(df_tiempo_por_persona['timestamp'])
df_tiempo_por_persona['time'] = df_tiempo_por_persona['DeltaTime'].dt.time

def make_delta(entry):
    h, m, s = str(entry).split(':')
    return datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))

df_tiempo_por_persona['DeltaTime'] = df_tiempo_por_persona['time'].apply(lambda entry: make_delta(entry))

In [785]:
df_tiempo_por_persona.head(5)

,person,timestamp,dia,mes,DeltaTime,time
0,4886f805,2018-05-18 00:11:59,18,5,00:11:59,00:11:59
1,ad93850f,2018-05-18 00:11:27,18,5,00:11:27,00:11:27
2,0297fc1e,2018-05-18 00:11:16,18,5,00:11:16,00:11:16
3,2d681dd8,2018-05-18 00:11:14,18,5,00:11:14,00:11:14
4,cccea85e,2018-05-18 00:11:09,18,5,00:11:09,00:11:09


In [786]:
df_horas_vp = df_tiempo_por_persona.groupby(['person','mes','dia']).agg({'DeltaTime': ['max','min']}).reset_index()
df_horas_vp['hora_por_dia'] = df_horas_vp['DeltaTime']['max'] - df_horas_vp['DeltaTime']['min']
df_horas_vp.head()

person mes dia DeltaTime          hora_por_dia
                          max      min             
0  00091926   5   3  22:09:59 22:08:35     00:01:24
1  00091926   5   5  23:45:04 03:43:24     20:01:40
2  00091926   5   6  02:05:38 02:04:55     00:00:43
3  00091926   5   7  00:37:20 00:31:13     00:06:07
4  00091926   5   8  23:42:50 23:30:44     00:12:06

In [787]:
df_horas_vp = df_horas_vp.groupby('person')['hora_por_dia'].sum().reset_index()
df_horas_vp.head()

,person,hora_por_dia
0,00091926,2 days 16:48:21
1,00091a7a,0 days 00:01:00
2,000ba417,0 days 04:23:53
3,000c79fe,0 days 00:07:09
4,000e4d9e,0 days 17:07:22


In [788]:
def segundos_totales(serie):
    return serie.total_seconds()


df_horas_vp['horas_totales'] = df_horas_vp['hora_por_dia'].apply(segundos_totales)
df_horas_vp.head()

,person,hora_por_dia,horas_totales
0,00091926,2 days 16:48:21,233301.0
1,00091a7a,0 days 00:01:00,60.0
2,000ba417,0 days 04:23:53,15833.0
3,000c79fe,0 days 00:07:09,429.0
4,000e4d9e,0 days 17:07:22,61642.0


In [789]:
df_horas_vp['horas_totales'] = df_horas_vp['horas_totales'] / 3600
df_horas_vp.head()

,person,hora_por_dia,horas_totales
0,00091926,2 days 16:48:21,64.805833
1,00091a7a,0 days 00:01:00,0.016667
2,000ba417,0 days 04:23:53,4.398056
3,000c79fe,0 days 00:07:09,0.119167
4,000e4d9e,0 days 17:07:22,17.122778


In [790]:
df_horas_vp = df_horas_vp.drop(columns=['hora_por_dia'])
df_horas_vp = df_horas_vp.rename(columns={'horas_totales':'horas_mirando_productos'})

In [791]:
df_horas_vp.head()

,person,horas_mirando_productos
0,00091926,64.805833
1,00091a7a,0.016667
2,000ba417,4.398056
3,000c79fe,0.119167
4,000e4d9e,17.122778


In [792]:
df_final = df_final.merge(df_horas_vp, on='person', how='left')

Lo mergeo al dataframe final

In [793]:
df_final = df_final.rename(columns={'most_visited_region':'region_persona'})
df_final.head()

,person,model,Cantidad_visitas,region_persona,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,b1f4dbf6,iPhone 5s,832,Sao Paulo,True,9.916010,True,352.695556
1,171e75cb,iPhone 6S,679,Espirito Santo,True,13.250000,True,213.320000
2,6f19cfd9,iPhone 6S Plus,622,Bahia,True,11.249071,True,317.761944
3,6abd2bf1,iPhone 6,603,Sao Paulo,True,9.829676,True,543.128611
4,9bf968c5,iPhone 6,558,Bahia,True,9.791096,True,276.193889


Nuevo feature: relacion conversion/checkout

# XGBoost

In [794]:
df_train = df_labels.merge(df_final, on='person', how='left')

In [795]:
df_train.head()

,person,label,model,Cantidad_visitas,region_persona,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,0566e9c1,0,iPhone 4S,4.0,Parana,False,5.567901,False,0.225556
1,6ec7ee77,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,iPhone 6,16.0,Sao Paulo,True,9.829676,True,33.452222
3,34728364,0,Lenovo Vibe K5,6.0,Sao Paulo,False,4.279805,True,0.571389
4,87ed62de,0,Samsung Galaxy J7,9.0,Santa Catarina,False,3.083333,True,0.390833


In [796]:
df_train = df_train.fillna(0)
df_train['busco_top_5_visitas'] = df_train['busco_top_5_visitas'].astype(int)
df_train['visito_mas_que_el_promedio'] = df_train['visito_mas_que_el_promedio'].astype(int)
df_train = df_train.drop(columns=['model','region_persona'])
df_train.head()

,person,label,Cantidad_visitas,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,0566e9c1,0,4.0,0,5.567901,0,0.225556
1,6ec7ee77,0,0.0,0,0.000000,0,0.000000
2,abe7a2fb,0,16.0,1,9.829676,1,33.452222
3,34728364,0,6.0,0,4.279805,1,0.571389
4,87ed62de,0,9.0,0,3.083333,1,0.390833


In [797]:
df_diff = pd.read_csv('diff.csv', low_memory = False)

In [798]:
df_diff= df_diff.merge(df_final,on='person',how='left')

In [799]:
df_diff.head()

,person,diff_mean,diff_max,session_count,model,Cantidad_visitas,region_persona,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,4886f805,0.058056,0.058056,1.0,Samsung Galaxy J7 Prime,4.0,Rio de Janeiro,False,5.040404,False,0.315278
1,ad93850f,0.095324,0.467500,6.0,iPhone 5s,17.0,Sao Paulo,True,9.916010,True,22.058056
2,0297fc1e,0.120095,0.678889,70.0,iPhone 6,213.0,Rio de Janeiro,True,9.286232,True,191.342500
3,2d681dd8,0.170093,0.376389,3.0,iPhone 7,6.0,Sao Paulo,True,8.858365,False,0.092222
4,cccea85e,0.152070,0.847778,42.0,Motorola Moto G4 Plus,252.0,Sao Paulo,False,9.077532,True,50.758611


In [800]:
df_FINAL = df_diff.drop(columns=['model','region_persona'])

In [801]:
df_FINAL = df_labels.merge(df_FINAL, on='person', how='left')

In [802]:
df_FINAL.head()

,person,label,diff_mean,diff_max,session_count,Cantidad_visitas,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,0566e9c1,0,0.059618,0.247778,8.0,4.0,False,5.567901,False,0.225556
1,6ec7ee77,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,0.095119,0.346944,21.0,16.0,True,9.829676,True,33.452222
3,34728364,0,0.043148,0.080556,3.0,6.0,False,4.279805,True,0.571389
4,87ed62de,0,0.155278,0.172222,2.0,9.0,False,3.083333,True,0.390833


In [803]:
df_FINAL = df_FINAL.fillna(0)
df_FINAL['busco_top_5_visitas'] = df_FINAL['busco_top_5_visitas'].astype(int)
df_FINAL['visito_mas_que_el_promedio'] = df_FINAL['visito_mas_que_el_promedio'].astype(int)

In [804]:
X, y = df_FINAL.iloc[:,2:],df_FINAL.iloc[:,1]

In [805]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [806]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

In [807]:
my_classifier1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [808]:
predicciones = my_classifier1.predict_proba(X_test)[:,1]

In [809]:
predicciones = pd.Series(predicciones)

In [810]:
df_predicciones = predicciones.to_frame()
df_predicciones[0].shape

(3883,)

In [811]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.6208698019773901

In [813]:
persons = df_labels['person']
persons_to_predict = (df_original.loc[~df_original['person'].isin(persons)])['person'].drop_duplicates().to_frame()
X_predict = persons_to_predict.merge(df_FINAL, on= 'person', how= 'left')
X_predict = X_predict.drop(['person'], axis=1)

In [814]:
X_predict = X_predict.fillna(0)
X_predict['busco_top_5_visitas'] = X_predict['busco_top_5_visitas'].astype(int)
X_predict['visito_mas_que_el_promedio'] = X_predict['visito_mas_que_el_promedio'].astype(int)

In [815]:
entrie = my_classifier1.predict_proba(X_predict)[:,1]

ValueError: feature_names mismatch: ['diff_mean', 'diff_max', 'session_count', 'Cantidad_visitas', 'busco_top_5_visitas', 'Promedio_visitas_producto', 'visito_mas_que_el_promedio', 'horas_mirando_productos'] ['label', 'diff_mean', 'diff_max', 'session_count', 'Cantidad_visitas', 'busco_top_5_visitas', 'Promedio_visitas_producto', 'visito_mas_que_el_promedio', 'horas_mirando_productos']
training data did not have the following fields: label

In [816]:
X_predict

,label,diff_mean,diff_max,session_count,Cantidad_visitas,busco_top_5_visitas,Promedio_visitas_producto,visito_mas_que_el_promedio,horas_mirando_productos
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
5,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
6,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
7,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
8,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
9,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
